In [1]:
# If needed (first run):
# !pip install xarray netCDF4 h5py numpy pandas

import os, math, json
import numpy as np
import pandas as pd
import h5py, xarray as xr
from pathlib import Path
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F


In [47]:
BASE = Path(r"C:\Users\jr80fd\swin_test")
INSAR_H5 = BASE / "timeseries_ERA5_ramp_demErr.h5"

W3RA_DIR = BASE / "w3ra"
geom_path = Path(r"C:\Users\JR80FD\swin_test\inputs\geometryGeo.h5")


print(f"BASE: {BASE}")
print(f"INSAR_H5: {INSAR_H5}")      

print(f"W3RA_DIR: {W3RA_DIR}")


BASE: C:\Users\jr80fd\swin_test
INSAR_H5: C:\Users\jr80fd\swin_test\timeseries_ERA5_ramp_demErr.h5
W3RA_DIR: C:\Users\jr80fd\swin_test\w3ra


In [3]:
import h5py
from pathlib import Path


def walk_h5(fn):
    with h5py.File(fn, 'r') as f:
        print("HDF5 file:", fn)
        def _walk(name, obj):
            if isinstance(obj, h5py.Dataset):
                print(f"[DATASET] {name}  shape={obj.shape}  dtype={obj.dtype}")
                for k,v in obj.attrs.items():
                    if k in ("rows","cols","length","mintpy.meta"):  # common MintPy attrs
                        print(f"   attr {k}: {v}")
            elif isinstance(obj, h5py.Group):
                print(f"[GROUP]   {name}")
        f.visititems(_walk)


In [4]:

walk_h5(INSAR_H5)


HDF5 file: C:\Users\jr80fd\swin_test\timeseries_ERA5_ramp_demErr.h5
[DATASET] bperp  shape=(156,)  dtype=float32
[DATASET] date  shape=(156,)  dtype=|S8
[DATASET] timeseries  shape=(156, 2939, 3713)  dtype=float32


# 1) Inspect the InSAR HDF5 structure

In [6]:
def summarize_h5(path):
    with h5py.File(path, "r") as f:
        print(f"File: {path}")
        print("Top-level keys:", list(f.keys()))
        for k in f.keys():
            obj = f[k]
            if isinstance(obj, h5py.Dataset):
                print(f"  Dataset: {k:18s} shape={obj.shape} dtype={obj.dtype}")
                if obj.attrs:
                    a = {k2: obj.attrs[k2] for k2 in obj.attrs.keys()}
                    print("    attrs keys:", list(a.keys())[:8])
            else:
                print(f"  Group:   {k}")
        # common MintPy fields
        if "date" in f:
            raw = f["date"][:]
            dates = [d.decode() if isinstance(d, (bytes, np.bytes_)) else str(d) for d in raw]
            print(f"\nFound {len(dates)} dates. First/last: {dates[:3]} ... {dates[-3:]}")
        if "timeseries" in f:
            print("timeseries shape:", f["timeseries"].shape)


In [7]:

summarize_h5(INSAR_H5)


File: C:\Users\jr80fd\swin_test\timeseries_ERA5_ramp_demErr.h5
Top-level keys: ['bperp', 'date', 'timeseries']
  Dataset: bperp              shape=(156,) dtype=float32
  Dataset: date               shape=(156,) dtype=|S8
  Dataset: timeseries         shape=(156, 2939, 3713) dtype=float32

Found 156 dates. First/last: ['20170610', '20170622', '20170704'] ... ['20220912', '20221006', '20221018']
timeseries shape: (156, 2939, 3713)


# 2) Load InSAR as an xarray.DataArray [time, y, x]

In [8]:
def load_insar_h5_as_xarray(path: Path) -> xr.DataArray:
    with h5py.File(path, "r") as f:
        # dates
        if "date" not in f:
            raise ValueError("No 'date' dataset found in HDF5.")
        raw = f["date"][:]
        dates = [d.decode() if isinstance(d, (bytes, np.bytes_)) else str(d) for d in raw]
        # MintPy often uses YYYYMMDD
        time = pd.to_datetime(dates, format="%Y%m%d", errors="coerce")
        # data
        ts = f["timeseries"][:]  # either (T,H,W) or (H,W,T)
        if ts.ndim != 3:
            raise ValueError("Expected 3D timeseries.")
        # infer axis order
        T, H, W = None, None, None
        if ts.shape[0] == len(time):        # (T,H,W)
            T, H, W = ts.shape
            data = ts
        elif ts.shape[-1] == len(time):     # (H,W,T)
            H, W, T = ts.shape
            data = np.transpose(ts, (2,0,1))   # -> (T,H,W)
        else:
            raise ValueError("Could not infer time axis. Check shapes/dates.")

        # build y/x coords (try attributes, else pixel indices)
        H5_attrs = f["timeseries"].attrs
        def get_attr(name, default=None):
            return H5_attrs[name] if name in H5_attrs else default

        # MintPy geocoded often has: X_FIRST, Y_FIRST, X_STEP, Y_STEP (lon/lat grid)
        x_first = get_attr("X_FIRST", None)
        y_first = get_attr("Y_FIRST", None)
        x_step  = get_attr("X_STEP", None)
        y_step  = get_attr("Y_STEP", None)
        if None not in (x_first, y_first, x_step, y_step):
            x = x_first + np.arange(W) * x_step
            y = y_first + np.arange(H) * y_step
        else:
            # fallback pixel indices
            x = np.arange(W)
            y = np.arange(H)

    da = xr.DataArray(
        data,
        dims=("time","y","x"),
        coords={"time": time, "y": y, "x": x},
        name="insar_deformation"
    )
    return da


In [9]:

insar_da = load_insar_h5_as_xarray(INSAR_H5)
insar_da


<xarray.DataArray 'insar_deformation' (time: 156, y: 2939, x: 3713)> Size: 7GB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2017-06-10 2017-06-22 ... 2022-10-18
  * y        (y) int32 12kB 0 1 2 3 4 5 6 ... 2932 2933 2934 2935 2936 2937 2938
  * x        (x) int32 15kB 0 1 2 3 4 5 6 ... 3706 3707 3708 3709 3710 3711 3712

# 3) Inspect W3RA NetCDFs and pick variable names

In [14]:
import xarray as xr
from pathlib import Path


for name in ["W3RA_2010_2024_.nc", "W3RA_2010_2024.nc"]:
    p = W3RA_DIR / name
    try:
        ds = xr.open_dataset(p)
        print("\n---", name, "---")
        print("vars:", list(ds.data_vars)[:10])
        print("dims:", dict(ds.dims))
        if "time" in ds:
            print("time:", str(ds.time.values[:3]), "…", str(ds.time.values[-3:]), f"({ds.dims['time']} steps)")
        ds.close()
    except Exception as e:
        print("\n---", name, "---\n", e)



--- W3RA_2010_2024_.nc ---
vars: ['S0_EU_2010', 'Sd_EU_2010', 'Sg_EU_2010', 'Sr_EU_2010', 'Ss_EU_2010', 'Ssnow_EU_2010', 'Stot_EU_2010', 'S0_EU_2011', 'Sd_EU_2011', 'Sg_EU_2011']
dims: {'lat': 22, 'lon': 24, 'time_2010': 365, 'time_2011': 365, 'time_2012': 366, 'time_2013': 365, 'time_2014': 365, 'time_2015': 365, 'time_2016': 366, 'time_2017': 365, 'time_2018': 365, 'time_2019': 365, 'time_2020': 366, 'time_2021': 365, 'time_2022': 365, 'time_2023': 365, 'time_2024': 213}

--- W3RA_2010_2024.nc ---
vars: []
dims: {'lat': 22, 'lon': 24}


C:\Users\JR80FD\AppData\Local\Temp\ipykernel_16316\4106634407.py:11: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print("dims:", dict(ds.dims))
C:\Users\JR80FD\AppData\Local\Temp\ipykernel_16316\4106634407.py:11: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print("dims:", dict(ds.dims))


In [15]:
ds = xr.open_dataset(W3RA_DIR / "W3RA_2010_2024_.nc")
# Adjust names if different in your file:
name_map = {"Sg":"Sg", "Sd":"Sd", "S0":"S0"}
for k in list(ds.data_vars):
    kl = k.lower()
    if "sg" in kl: name_map["Sg"] = k
    if "sd" in kl: name_map["Sd"] = k
    if "s0" in kl or "stot0" in kl: name_map["S0"] = k

sg_all = ds[name_map["Sg"]]
sd_all = ds[name_map["Sd"]]
s0_all = ds[name_map["S0"]]
print("picked:", name_map)
print(sg_all.shape, sd_all.shape, s0_all.shape, sg_all.dims)


picked: {'Sg': 'Sg_EU_2024', 'Sd': 'Sd_EU_2024', 'S0': 'S0_EU_2024'}
(213, 22, 24) (213, 22, 24) (213, 22, 24) ('time_2024', 'lat', 'lon')


Stitch W3RA years into continuous time series

In [16]:
import xarray as xr
import pandas as pd
import re
from pathlib import Path



def stitch_years(ds, base_prefix):
    """
    Collect variables like f'{base_prefix}_{YYYY}' each with dim 'time_YYYY',
    rename that dim to 'time', cast to datetime, then concat over time.
    Returns a single DataArray with dims (time, lat, lon).
    """
    pats = []
    for k in ds.data_vars:
        m = re.fullmatch(fr"{re.escape(base_prefix)}_(\d{{4}})", k)
        if m: pats.append((int(m.group(1)), k))
    if not pats:
        raise ValueError(f"No yearly vars found for prefix {base_prefix}")
    pats.sort()  # by year
    chunks = []
    for year, var in pats:
        da = ds[var]
        time_dim = f"time_{year}"
        if time_dim not in da.dims:
            raise ValueError(f"{var} missing dim {time_dim}")
        da = da.rename({time_dim: "time"})
        # ensure datetime64
        da = da.assign_coords(time=pd.to_datetime(da['time'].values))
        chunks.append(da)
    out = xr.concat(chunks, dim="time")
    return out


In [17]:

sg_all = stitch_years(ds, "Sg_EU")
sd_all = stitch_years(ds, "Sd_EU")
s0_all = stitch_years(ds, "S0_EU")
print(sg_all.shape, sd_all.shape, s0_all.shape, sg_all.dims)


(5326, 22, 24) (5326, 22, 24) (5326, 22, 24) ('time', 'lat', 'lon')


# 4) Align the time window to your InSAR 
(2017-06-10 → 2022-10-18) and monthly resample

In [18]:
# InSAR time range from your HDF5 summary:
tmin_insar = pd.Timestamp("2017-06-01")   # round to month for resample
tmax_insar = pd.Timestamp("2022-10-31")

# Monthly means
sg_m = sg_all.resample(time="MS").mean()
sd_m = sd_all.resample(time="MS").mean()
s0_m = s0_all.resample(time="MS").mean()

# Subset to common window
sg_m = sg_m.sel(time=slice(tmin_insar, tmax_insar))
sd_m = sd_m.sel(time=slice(tmin_insar, tmax_insar))
s0_m = s0_m.sel(time=slice(tmin_insar, tmax_insar))
print("W3RA monthly shapes:", sg_m.shape, sd_m.shape, s0_m.shape)


W3RA monthly shapes: (65, 22, 24) (65, 22, 24) (65, 22, 24)


# 5) Build InSAR xarray with lat/lon from HDF5 attrs (1D grid)

In [19]:
import h5py, numpy as np, xarray as xr, pandas as pd
from pathlib import Path


with h5py.File(INSAR_H5, "r") as f:
    ts = f["timeseries"][:]             # (T,H,W)
    raw_dates = [d.decode() for d in f["date"][:]]
    time = pd.to_datetime(raw_dates, format="%Y%m%d")
    attrs = f["timeseries"].attrs
    X_FIRST = attrs.get("X_FIRST", None)
    Y_FIRST = attrs.get("Y_FIRST", None)
    X_STEP  = attrs.get("X_STEP",  None)
    Y_STEP  = attrs.get("Y_STEP",  None)

T,H,W = ts.shape
insar_da = xr.DataArray(ts, dims=("time","y","x"),
                        coords={"time": time,
                                "y": np.arange(H),
                                "x": np.arange(W)},
                        name="insar_deformation")

# If geocoded attrs are present, add lon/lat 1D coords
if None not in (X_FIRST, X_STEP, Y_FIRST, Y_STEP):
    lon_1d = X_FIRST + np.arange(W) * X_STEP
    lat_1d = Y_FIRST + np.arange(H) * Y_STEP
    insar_da = insar_da.assign_coords(lon=("x", lon_1d), lat=("y", lat_1d))

# Resample to monthly means and subset
insar_m = insar_da.resample(time="MS").mean().sel(time=slice(tmin_insar, tmax_insar))
print("InSAR monthly shape:", insar_m.shape)


InSAR monthly shape: (65, 2939, 3713)


In [20]:
import h5py
from pathlib import Path


with h5py.File(INSAR_H5, "r") as g:
    print("Top-level datasets:", list(g.keys()))           # e.g., ['latitude','longitude',...]
    for k in ["latitude","longitude","y","x","azimuthCoord","rangeCoord"]:
        if k in g:
            print(k, g[k].shape, g[k].dtype)
    # global attrs can carry projection info (EPSG/UTM zone)
    print("attrs:", {a: g.attrs[a] for a in g.attrs.keys()})


Top-level datasets: ['bperp', 'date', 'timeseries']
attrs: {'ALOOKS': '4', 'ANTENNA_SIDE': '-1', 'AZIMUTH_PIXEL_SIZE': '56.4', 'BANDS': '1', 'CENTER_LINE_UTC': '19151.797495', 'DATA_TYPE': 'float32', 'DATE12': '170610-170622', 'EARTH_RADIUS': '6367896.9087', 'END_DATE': '20221018', 'EPSG': '32633', 'FILE_LENGTH': '2945', 'FILE_PATH': '/mnt/data/aoi_3_02_bologna/MintPy/timeseries_ERA5_ramp_demErr.h5', 'FILE_TYPE': 'timeseries', 'HEADING': '-165.9793419', 'HEIGHT': '701867.9117', 'INTERLEAVE': 'BSQ', 'LAT_REF1': '5011480.0', 'LAT_REF2': '5011480.0', 'LAT_REF3': '4775880.0', 'LAT_REF4': '4775880.0', 'LENGTH': '2939', 'LON_REF1': '413880.0', 'LON_REF2': '116360.0', 'LON_REF3': '413880.0', 'LON_REF4': '116360.0', 'NO_DATA_VALUE': '0.0', 'NoDataValue': '0.0', 'ORBIT_DIRECTION': 'DESCENDING', 'PLATFORM': 'Sen', 'PROCESSOR': 'hyp3', 'PROJECT_NAME': 'mintpy_config', 'P_BASELINE_BOTTOM_HDR': '-3.725', 'P_BASELINE_TOP_HDR': '-3.725', 'RANGE_PIXEL_SIZE': '46.0', 'REF_DATE': '20220103', 'REF_LAT': 

In [21]:
import h5py
with h5py.File(INSAR_H5, "r") as f:
    A = f["timeseries"].attrs
    print({k: A[k] for k in ["EPSG","X_FIRST","X_STEP","X_UNIT","Y_FIRST","Y_STEP","Y_UNIT"] if k in A})


{}


In [22]:
import numpy as np
import h5py
from pyproj import Transformer

# ------------- get UTM grid from geometryGeo.h5 -------------
geom_path = r"C:\Users\JR80FD\swin_test\inputs\geometryGeo.h5"
with h5py.File(geom_path, "r") as g:
    EPSG = int(g.attrs.get("EPSG", 32633))           # 32633 = UTM 33N
    X_FIRST = float(g.attrs["X_FIRST"])              # meters
    X_STEP  = float(g.attrs["X_STEP"])               # meters
    Y_FIRST = float(g.attrs["Y_FIRST"])              # meters
    Y_STEP  = float(g.attrs["Y_STEP"])               # meters

H = insar_da.sizes["y"]
W = insar_da.sizes["x"]

# Easting/Northing 1D
x_e = X_FIRST + np.arange(W) * X_STEP                # shape (W,)
y_n = Y_FIRST + np.arange(H) * Y_STEP                # shape (H,)

# 2D mesh in UTM
XX, YY = np.meshgrid(x_e, y_n)                       # (H,W)

# Transform UTM -> lon/lat
to_ll = Transformer.from_crs(EPSG, 4326, always_xy=True)
lon2d, lat2d = to_ll.transform(XX, YY)               # float64 arrays

# keep memory sane
lon2d = lon2d.astype("float32")
lat2d = lat2d.astype("float32")

# Attach as coords to your existing insar_da
insar_da = insar_da.assign_coords(lat=(("y","x"), lat2d),
                                  lon=(("y","x"), lon2d))

# Monthly resample + subset (reuse your existing time window if you defined it)
insar_m = insar_da.resample(time="MS").mean()
insar_m = insar_m.sel(time=slice("2017-06-01", "2022-10-31"))

print("InSAR monthly:", insar_m.shape, "| coords now:", [c for c in insar_m.coords if c in ("lat","lon")])


InSAR monthly: (65, 2939, 3713) | coords now: ['lat', 'lon']


In [23]:
print(INSAR_H5)

C:\Users\jr80fd\swin_test\timeseries_ERA5_ramp_demErr.h5


# 6) Interpolate W3RA → InSAR grid (bilinear)

W3RA is daily (2010–2024).

InSAR is irregular (6/12 days) but you resampled it to monthly means (156 steps for 2017–2022).
Both datasets aligned as 4D tensors

In [24]:
# Time-align first
insar_m, sg_m = xr.align(insar_m, sg_m, join="inner")
_, sd_m = xr.align(insar_m, sd_m, join="inner")
_, s0_m = xr.align(insar_m, s0_m, join="inner")

# Bilinear interpolation onto the 2D lon/lat of InSAR
sg_on_insar = sg_m.interp(lat=insar_m["lat"], lon=insar_m["lon"], method="linear")
sd_on_insar = sd_m.interp(lat=insar_m["lat"], lon=insar_m["lon"], method="linear")
s0_on_insar = s0_m.interp(lat=insar_m["lat"], lon=insar_m["lon"], method="linear")

print("Regridded:", sg_on_insar.shape, sd_on_insar.shape, s0_on_insar.shape)


Regridded: (65, 2939, 3713) (65, 2939, 3713) (65, 2939, 3713)


In [ ]:
# Paths
out_insar = BASE / "insar_aligned.nc"
out_w3ra  = BASE / "w3ra_aligned.nc"

# Save InSAR
insar_m.to_netcdf(out_insar)

# Combine W3RA (stack channels into one Dataset)
xr.Dataset({
    "Sg": sg_m,
    "Sd": sd_m,
    "S0": s0_m,
}).to_netcdf(out_w3ra)

print("Saved:", out_insar, out_w3ra)


Saved: C:\Users\jr80fd\swin_test\insar_aligned.nc C:\Users\jr80fd\swin_test\w3ra_aligned.nc


In [26]:
print(insar_m.shape, sg_on_insar.shape, sd_on_insar.shape, s0_on_insar.shape)
print(insar_m.time.equals(sg_on_insar.time),
      insar_m.y.equals(sg_on_insar.y),
      insar_m.x.equals(sg_on_insar.x))


(65, 2939, 3713) (65, 2939, 3713) (65, 2939, 3713) (65, 2939, 3713)
True True True


In [27]:
for da,name in [(insar_m,"InSAR"),(sg_on_insar,"Sg"),(sd_on_insar,"Sd"),(s0_on_insar,"S0")]:
    print(name, "NaN% =", float(da.isnull().mean().values)*100)


InSAR NaN% = 15.384615384615385
Sg NaN% = 50.910693573896445
Sd NaN% = 50.910693573896445
S0 NaN% = 50.910693573896445


In [28]:
print(pd.Index(insar_m.time.values).difference(sg_on_insar.time.values))


DatetimeIndex([], dtype='datetime64[ns]', freq=None)


In [29]:
i,j = insar_m.sizes['y']//2, insar_m.sizes['x']//2
print("center pixel:", float(insar_m.isel(y=i,x=j).std()), 
      float(sg_on_insar.isel(y=i,x=j).std()))


center pixel: 0.01842602901160717 nan


c:\Users\JR80FD\.conda\envs\datasci_env\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [30]:
lat,lon = float(insar_m.lat.isel(y=i,x=j)), float(insar_m.lon.isel(y=i,x=j))
print("lat/lon @center:", lat, lon)


lat/lon @center: 44.820499420166016 12.223923683166504


In [ ]:
# Paths
out_insar = BASE / "insar_aligned.nc"
out_w3ra  = BASE / "w3ra_aligned.nc"

# Save InSAR
insar_m.to_netcdf(out_insar)

# Combine W3RA (stack channels into one Dataset)
xr.Dataset({
    "Sg": sg_m,
    "Sd": sd_m,
    "S0": s0_m,
}).to_netcdf(out_w3ra)

print("Saved:", out_insar, out_w3ra)


In [ ]:
import os, math, json, time
from pathlib import Path

import xarray as xr
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
torch.manual_seed(SEED); np.random.seed(SEED)

BASE = Path(BASE)  # <- respects your existing BASE (don’t overwrite)
insar_nc = BASE / "insar_aligned.nc"
w3ra_nc  = BASE / "w3ra_aligned.nc"

assert insar_nc.exists() and w3ra_nc.exists(), "NetCDF files not found."

def find_single_datavar(ds: xr.Dataset):
    # return the only/first data variable name
    vars_ = list(ds.data_vars)
    if not vars_:
        raise ValueError("No data variables in Dataset.")
    # prefer common names if present
    for k in ["uz","u","defo","disp","unw","vel","def"]:
        if k in vars_:
            return k
    return vars_[0]


In [33]:
# If you ever did f = h5py.File(...), make sure it's closed
try:
    f.close()
except Exception:
    pass


Detach xarray object from any backing file - force data into RAM 

In [ ]:
INSAR_H5_ts = insar_m  
W3RA_H5_ts  = BASE / "w3ra_aligned.h5"    # contains Sg, Sd, S0


In [42]:



# ---- model/data config ----
WIN_T = 12             # months per training window (receptive field in time)
PATCH = (1, 128, 128)  # (t,h,w) patch fed to model; t must equal WIN_T for this script
STRIDE = (1, 128, 128) # sampling stride for patches
BATCH_SIZE = 2
NUM_EPOCHS = 5
LR = 2e-4
DEVICE = "cpu"   # "cuda" if torch.cuda.is_available() else "cpu"


In [43]:
import h5py, numpy as np, torch
from torch.utils.data import Dataset, DataLoader
'''
USAGE: 
train_ds = InSAR2W3RA(INSAR_H5, W3RA_H5, win_t=WIN_T, patch_sz=PATCH, stride=STRIDE, split="train")
val_ds   = InSAR2W3RA(INSAR_H5, W3RA_H5, win_t=WIN_T, patch_sz=PATCH, stride=STRIDE, split="val")

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
len(train_ds), len(val_ds)
'''

class InSAR2W3RA(Dataset):
    def __init__(self, insar_h5, w3ra_h5, win_t=12, patch_sz=(1,128,128), stride=(1,128,128),
                 split="train", val_frac=0.1, test_frac=0.1, zscore=True, seed=0):
        self.f_in = h5py.File(insar_h5, "r")
        self.f_w3 = h5py.File(w3ra_h5,  "r")
        self.D_in = self.f_in["InSAR"]        # (T,H,W)
        self.D0 = self.f_w3["S0"]             # (T,H,W)
        self.Dd = self.f_w3["Sd"]
        self.Dg = self.f_w3["Sg"]

        T,H,W = self.D_in.shape
        self.win_t = win_t
        self.pz = patch_sz
        self.stride = stride
        assert patch_sz[0] in (1, win_t), "Time patch must be 1 or WIN_T; use WIN_T here."
        assert win_t <= T

        # build list of (t0,y0,x0) windows
        t_idx = np.arange(0, T - win_t + 1, stride[0])
        y_idx = np.arange(0, H - patch_sz[1] + 1, stride[1])
        x_idx = np.arange(0, W - patch_sz[2] + 1, stride[2])
        idx = np.stack(np.meshgrid(t_idx, y_idx, x_idx, indexing="ij"), -1).reshape(-1,3)

        # split by time (keeps spatial leakage low)
        n_t = len(t_idx)
        n_val = max(1,int(n_t*val_frac))
        n_test= max(1,int(n_t*test_frac))
        rng = np.random.default_rng(seed)
        # choose complete time starts for val/test
        t_perm = rng.permutation(n_t)
        t_val  = set(t_idx[t_perm[:n_val]])
        t_test = set(t_idx[t_perm[n_val:n_val+n_test]])

        if split=="train":
            mask = np.array([t not in t_val and t not in t_test for t,_,_ in idx])
        elif split=="val":
            mask = np.array([t in t_val for t,_,_ in idx])
        else:
            mask = np.array([t in t_test for t,_,_ in idx])

        self.samples = idx[mask]

        # compute z-score statistics (approx: sample 200 patches)
        self.zscore = zscore
        if zscore:
            self.x_mean, self.x_std, self.y_mean, self.y_std = self._estimate_stats(rng, nsamp=200)
        else:
            self.x_mean = self.x_std = self.y_mean = self.y_std = None

    def _estimate_stats(self, rng, nsamp=200):
        xs, ys = [], []
        for _ in range(min(nsamp, len(self.samples))):
            t,y,x = self.samples[rng.integers(len(self.samples))]
            s = slice(t, t+self.win_t)
            yy = slice(y, y+self.pz[1]); xx = slice(x, x+self.pz[2])
            xin = self.D_in[s, yy, xx]       # (T,H,W)
            yout = np.stack([self.D0[s, yy, xx],
                             self.Dd[s, yy, xx],
                             self.Dg[s, yy, xx]], axis=0)   # (3,T,H,W)
            xs.append(np.nan_to_num(xin, nan=0.0))
            ys.append(np.nan_to_num(yout, nan=0.0))
        xcat = np.concatenate([u.reshape(-1,1) for u in xs],0)
        ycat = np.concatenate([u.reshape(-1,1) for u in ys],0)
        x_mean, x_std = float(np.mean(xcat)), float(np.std(xcat)+1e-6)
        y_mean, y_std = float(np.mean(ycat)), float(np.std(ycat)+1e-6)
        return x_mean, x_std, y_mean, y_std

    def __len__(self): return len(self.samples)

    def __getitem__(self, i):
        t,y,x = self.samples[i]
        s = slice(t, t+self.win_t)
        yy = slice(y, y+self.pz[1]); xx = slice(x, x+self.pz[2])

        xin  = self.D_in[s, yy, xx].astype(np.float32)         # (T,H,W)
        yout = np.stack([ self.D0[s,yy,xx], self.Dd[s,yy,xx], self.Dg[s,yy,xx] ],
                        axis=0).astype(np.float32)             # (3,T,H,W)

        # masks for NaN
        m = ~np.isnan(yout)                                    # (3,T,H,W)
        xin  = np.nan_to_num(xin,  nan=0.0)
        yout = np.nan_to_num(yout, nan=0.0)

        if self.zscore:
            xin  = (xin  - self.x_mean)/self.x_std
            yout = (yout - self.y_mean)/self.y_std

        x_t = torch.from_numpy(xin).unsqueeze(0)               # [1,T,H,W]
        y_t = torch.from_numpy(yout)                           # [3,T,H,W]
        m_t = torch.from_numpy(m.astype(np.float32))           # [3,T,H,W]
        return x_t, y_t, m_t


In [48]:
import xarray as xr
from pathlib import Path


p = BASE / "w3ra_aligned.nc"
try:
    ds = xr.open_dataset(p)
    print("\n---", name, "---")
    print("vars:", list(ds.data_vars)[:10])
    print("dims:", dict(ds.dims))
    if "time" in ds:
        print("time:", str(ds.time.values[:3]), "…", str(ds.time.values[-3:]), f"({ds.dims['time']} steps)")
    ds.close()
except Exception as e:
    print("\n---", name, "---\n", e)



--- S0 ---
vars: ['Sg', 'Sd', 'S0']
dims: {'lat': 22, 'lon': 24, 'time': 65}
time: ['2017-06-01T00:00:00.000000000' '2017-07-01T00:00:00.000000000'
 '2017-08-01T00:00:00.000000000'] … ['2022-08-01T00:00:00.000000000' '2022-09-01T00:00:00.000000000'
 '2022-10-01T00:00:00.000000000'] (65 steps)


C:\Users\JR80FD\AppData\Local\Temp\ipykernel_16316\2079874011.py:10: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print("dims:", dict(ds.dims))
C:\Users\JR80FD\AppData\Local\Temp\ipykernel_16316\2079874011.py:12: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print("time:", str(ds.time.values[:3]), "…", str(ds.time.values[-3:]), f"({ds.dims['time']} steps)")


In [44]:
from torch.utils.data import DataLoader


train_ds = InSAR2W3RA(INSAR_H5_ts, W3RA_H5_ts, win_t=WIN_T, patch_sz=PATCH, stride=STRIDE, split="train")
val_ds   = InSAR2W3RA(INSAR_H5_ts, W3RA_H5_ts, win_t=WIN_T, patch_sz=PATCH, stride=STRIDE, split="val")


train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE,
                      shuffle=True, num_workers=0,  # <-- important
                      pin_memory=False, persistent_workers=False)

val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE,
                      shuffle=False, num_workers=0, # <-- important
                      pin_memory=False, persistent_workers=False)



len(train_ds), len(val_ds)

KeyError: "Unable to synchronously open object (object 'S0' doesn't exist)"

In [46]:
import torch.nn as nn
import torch.nn.functional as F

def window_partition_3d(x, win):  # x: [B,T,H,W,C]
    WT,WH,WW = win
    B,T,H,W,C = x.shape
    assert T%WT==0 and H%WH==0 and W%WW==0
    x = x.view(B, T//WT, WT, H//WH, WH, W//WW, WW, C)
    x = x.permute(0,1,3,5,2,4,6,7).contiguous()               # [B, nWT, nWH, nWW, WT, WH, WW, C]
    return x.view(-1, WT*WH*WW, C)                            # [B*nWins, tokens, C]

def window_unpartition_3d(windows, win, B, T, H, W, C):
    WT,WH,WW = win
    nWT, nWH, nWW = T//WT, H//WH, W//WW
    x = windows.view(B, nWT, nWH, nWW, WT, WH, WW, C)
    x = x.permute(0,1,4,2,5,3,6,7).contiguous()
    return x.view(B, T, H, W, C)

class MLP(nn.Module):
    def __init__(self, dim, mlp_ratio=4):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim*mlp_ratio)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(dim*mlp_ratio, dim)
    def forward(self, x): return self.fc2(self.act(self.fc1(x)))

class SwinBlock3D(nn.Module):
    def __init__(self, dim, num_heads=4, window_size=(2,4,4), shift=False):
        super().__init__()
        self.dim = dim
        self.win = window_size
        self.shift = shift
        self.norm1 = nn.LayerNorm(dim)
        self.attn  = nn.MultiheadAttention(dim, num_heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp   = MLP(dim)

    def forward(self, x):               # x: [B,T,H,W,C]
        B,T,H,W,C = x.shape
        if self.shift:
            sT,sH,sW = self.win[0]//2, self.win[1]//2, self.win[2]//2
            x = torch.roll(x, shifts=(-sT,-sH,-sW), dims=(1,2,3))
        # partition → attn per window
        xw = window_partition_3d(x, self.win)               # [B*nWins, tokens, C]
        xw = self.norm1(xw)
        attn_out,_ = self.attn(xw, xw, xw)                  # window MSA
        xw = xw + attn_out
        xw = xw + self.mlp(self.norm2(xw))
        # unpartition
        x = window_unpartition_3d(xw, self.win, B,T,H,W,C)
        if self.shift:
            sT,sH,sW = self.win[0]//2, self.win[1]//2, self.win[2]//2
            x = torch.roll(x, shifts=(+sT,+sH,+sW), dims=(1,2,3))
        return x

class PatchEmbed3D(nn.Module):
    def __init__(self, in_ch=1, embed_dim=96, patch_size=(1,4,4)):
        super().__init__()
        self.proj = nn.Conv3d(in_ch, embed_dim, kernel_size=patch_size, stride=patch_size)
    def forward(self, x):               # [B,1,T,H,W]
        x = self.proj(x)                # [B,C,T',H',W']
        x = x.permute(0,2,3,4,1)        # → [B,T',H',W',C]
        return x

class PatchMerge3D(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.reduction = nn.Linear(dim*4, dim*2)
    def forward(self, x):               # [B,T,H,W,C]
        B,T,H,W,C = x.shape
        x = x.view(B,T, H//2,2, W//2,2, C).permute(0,1,2,4,3,5,6).contiguous()
        x = x.view(B,T, H//2, W//2, 4*C)
        return self.reduction(x)

class PatchExpand3D(nn.Module):
    def __init__(self, dim_out):
        super().__init__()
        self.dim_out = dim_out
    def forward(self, x):               # [B,T,H,W,Cin] → upsample H,W x2, reduce channels
        B,T,H,W,C = x.shape
        x = x.permute(0,4,1,2,3)        # [B,C,T,H,W]
        x = F.interpolate(x, scale_factor=(1,2,2), mode="trilinear", align_corners=False)
        x = nn.Conv3d(C, self.dim_out, kernel_size=1)(x)
        return x.permute(0,2,3,4,1)     # [B,T,H',W',Cout]

class SwinUNet3D(nn.Module):
    def __init__(self, in_ch=1, out_ch=3, embed_dim=96,
                 depths=(2,2,2), num_heads=(3,3,3),
                 window_size=(2,4,4), patch_size=(1,4,4)):
        super().__init__()
        self.patch_embed = PatchEmbed3D(in_ch, embed_dim, patch_size)
        C = embed_dim

        # encoder stages
        self.enc1 = nn.ModuleList([SwinBlock3D(C,   num_heads[0], window_size, shift=(i%2==1)) for i in range(depths[0])])
        self.pm1  = PatchMerge3D(C);   C *= 2
        self.enc2 = nn.ModuleList([SwinBlock3D(C,   num_heads[1], window_size, shift=(i%2==1)) for i in range(depths[1])])
        self.pm2  = PatchMerge3D(C);   C *= 2
        self.enc3 = nn.ModuleList([SwinBlock3D(C,   num_heads[2], window_size, shift=(i%2==1)) for i in range(depths[2])])

        # decoder
        self.up2  = PatchExpand3D(C//2) # to match skip2
        self.dec2 = nn.ModuleList([SwinBlock3D(C,   num_heads[1], window_size, shift=(i%2==1)) for i in range(1)])
        self.up1  = PatchExpand3D(C//4) # to match skip1
        self.dec1 = nn.ModuleList([SwinBlock3D(C//2,num_heads[0], window_size, shift=(i%2==1)) for i in range(1)])

        self.head = nn.Sequential(
            nn.LayerNorm(C//2),
            nn.Linear(C//2, out_ch)
        )

    def forward(self, x):               # x: [B,1,T,H,W]
        B = x.size(0)
        z = self.patch_embed(x)         # [B,T',H',W',C]
        # enc1
        for blk in self.enc1: z = blk(z)
        skip1 = z
        z = self.pm1(z)                 # down x2 (H,W), Cx2
        # enc2
        for blk in self.enc2: z = blk(z)
        skip2 = z
        z = self.pm2(z)                 # down x2 again
        # enc3 (bottleneck)
        for blk in self.enc3: z = blk(z)
        # up2
        z = self.up2(z)                 # up x2
        z = torch.cat([z, skip2], dim=-1)
        for blk in self.dec2: z = blk(z)
        # up1
        z = self.up1(z)                 # up x2
        z = torch.cat([z, skip1], dim=-1)
        for blk in self.dec1: z = blk(z)
        # head linear per-token → channels
        B,T,H,W,C = z.shape
        z = self.head(z).view(B,T,H,W,-1).permute(0,4,1,2,3)  # [B,out_ch,T,H,W]
        return z


In [47]:
class MaskedMAE(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, pred, tgt, mask):
        # pred,tgt: [B,3,T,H,W]; mask: [B,3,T,H,W] (1=valid)
        valid = mask>0.5
        diff = torch.abs(pred - tgt)
        diff = diff[valid]
        return diff.mean() if diff.numel() else (pred-tgt).abs().mean()


In [51]:
import torch

DEVICE = "cpu"   # force CPU
print("Using", DEVICE)


Using cpu


In [ ]:
model = SwinUNet3D(in_ch=1, out_ch=3, embed_dim=64,
                   depths=(2,2,2), num_heads=(2,4,4),
                   window_size=(2,4,4), patch_size=(1,4,4)).to(DEVICE)

opt  = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)
loss_fn = MaskedMAE()

def step(dl, train=True):
    model.train(train)
    tot, n = 0.0, 0
    for x,y,m in dl:
        x = x.to(DEVICE)                 # [B,1,T,H,W]
        y = y.to(DEVICE)                 # [B,3,T,H,W]
        m = m.to(DEVICE)                 # [B,3,T,H,W]
        if train: opt.zero_grad()
        yhat = model(x)
        loss = loss_fn(yhat, y, m)
        if train:
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
        tot += float(loss.detach().cpu())
        n += 1
    return tot/max(1,n)

for epoch in range(1, NUM_EPOCHS+1):
    tr = step(train_dl, True)
    va = step(val_dl,   False)
    print(f"Epoch {epoch:02d} | train {tr:.4f} | val {va:.4f}")


RuntimeError: The size of tensor a (32) must match the size of tensor b (128) at non-singleton dimension 4

: 

# 6) Build model tensors [B, C, T, H, W]

Input 
X: InSAR → [B, 1, T, H, W]

Output 
Y: stack W3RA layers → [B, 3, T, H, W]

We’ll also normalize (z-score) and keep the stats for de-normalization later.

In [ ]:
# stack W3RA channels in the order [Sg, Sd, S0]
w3ra_stack = xr.concat([sg_r, sd_r, s0_r], dim="channel").assign_coords(channel=["Sg","Sd","S0"])
print("W3RA stack:", w3ra_stack.shape, w3ra_stack.coords["channel"].values)

# Ensure matching dims order: (time, y, x)
insar_m = insar_m.transpose("time","y","x")
w3ra_stack = w3ra_stack.transpose("time","y","x")

T, H, W = insar_m.shape
C_out = 3

# to numpy
x_np = insar_m.values.astype(np.float32)                     # (T,H,W)
y_np = w3ra_stack.values.astype(np.float32)                  # (C_out,T,H,W)

# add batch and channel dimensions → [B, C, T, H, W]
x_t = torch.from_numpy(x_np).unsqueeze(0).unsqueeze(1)       # [1,1,T,H,W]
y_t = torch.from_numpy(y_np).unsqueeze(0)                    # [1,C_out,T,H,W]

print("X tensor:", x_t.shape, " Y tensor:", y_t.shape)

# ---- Normalization ----
# InSAR: single channel
x_mean = x_t.mean(); x_std = x_t.std().clamp_min(1e-6)
x_norm = (x_t - x_mean) / x_std

# W3RA per-channel stats
y_mean = y_t.mean(dim=(0,2,3,4), keepdim=True)
y_std  = y_t.std(dim=(0,2,3,4), keepdim=True).clamp_min(1e-6)
y_norm = (y_t - y_mean) / y_std

norm_stats = {"x_mean": float(x_mean), "x_std": float(x_std),
              "y_mean": y_mean.squeeze().tolist(), "y_std": y_std.squeeze().tolist()}
norm_stats


# 7) Minimal model (the “modified” Swin-lite U-Net)

Paste this once if you don’t already have it in the notebook.

In [ ]:
class SimpleSwinBlock3D(nn.Module):
    def __init__(self, dim, num_heads=4, mlp_ratio=4.0):
        super().__init__()
        self.dim = dim
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        hidden = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(nn.Linear(dim, hidden), nn.GELU(), nn.Linear(hidden, dim))

    def forward(self, x):  # [B,C,T,H,W]
        B, C, T, H, W = x.shape
        y = x.permute(0,2,3,4,1).contiguous().view(B, T*H*W, C)
        z = self.norm1(y)
        out,_ = self.attn(z,z,z)
        y = y + out
        y = y + self.mlp(self.norm2(y))
        return y.view(B,T,H,W,C).permute(0,4,1,2,3).contiguous()

class PatchEmbed3D(nn.Module):
    def __init__(self, in_ch=1, embed_dim=64, patch_size=(1,4,4)):
        super().__init__()
        self.proj = nn.Conv3d(in_ch, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.bn = nn.BatchNorm3d(embed_dim)
    def forward(self, x):
        return self.bn(self.proj(x))

class Swin3DNetMini(nn.Module):
    def __init__(self, in_ch=1, out_ch=3, embed_dim=64):
        super().__init__()
        self.patch = PatchEmbed3D(in_ch, embed_dim, patch_size=(1,4,4))
        self.enc1  = SimpleSwinBlock3D(embed_dim, num_heads=4)
        self.down  = nn.Conv3d(embed_dim, embed_dim*2, kernel_size=(1,2,2), stride=(1,2,2))
        self.enc2  = SimpleSwinBlock3D(embed_dim*2, num_heads=8)
        self.up    = nn.ConvTranspose3d(embed_dim*2, embed_dim, kernel_size=(1,2,2), stride=(1,2,2))
        self.fuse  = nn.Conv3d(embed_dim*2, embed_dim, kernel_size=1)
        self.head  = nn.Conv3d(embed_dim, out_ch, kernel_size=1)

    def forward(self, x):  # [B,1,T,H,W]
        B,_,T,H,W = x.shape
        z0 = self.patch(x)           # [B,D,T,H/4,W/4]
        e1 = self.enc1(z0)
        e2i= self.down(e1)           # [B,2D,T,H/8,W/8]
        e2 = self.enc2(e2i)
        up = self.up(e2)             # [B,D,T,H/4,W/4]
        cat= torch.cat([up, e1], dim=1)
        fz = self.fuse(cat)
        y  = self.head(fz)           # [B,out_ch,T,H/4,W/4]
        # back to original size
        y  = F.interpolate(y, size=(T,H,W), mode="trilinear", align_corners=False)
        return y


# 8) Train for a few epochs (sanity)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Swin3DNetMini(in_ch=1, out_ch=3, embed_dim=64).to(device)

x_batch = x_norm.to(device)  # [1,1,T,H,W]
y_batch = y_norm.to(device)  # [1,3,T,H,W]

criterion = nn.L1Loss()  # MAE (you can switch to MSE)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-5)

loss_history = []
model.train()
for epoch in range(10):   # start small to test pipeline
    optimizer.zero_grad()
    y_pred = model(x_batch)
    loss = criterion(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    print(f"epoch {epoch+1:02d} | loss {loss.item():.5f}")


In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_history); plt.xlabel("epoch"); plt.ylabel("train loss"); plt.show()


# 9) De-normalize and inspect one pixel time series + lag

In [ ]:
model.eval()
with torch.no_grad():
    y_hat = model(x_batch).cpu()    # [1,3,T,H,W]

# de-normalize
y_hat_dn = y_hat * y_std + y_mean   # back to original units

# center pixel
_, C, T, H, W = y_hat.shape
i, j = H//2, W//2
names = ["Sg","Sd","S0"]

def best_lag(yt, yp, max_lag=6):
    lags = np.arange(-max_lag, max_lag+1)
    corrs = []
    for L in lags:
        if L>0: a,b = yt[L:], yp[:-L]
        elif L<0: a,b = yt[:L], yp[-L:]
        else: a,b = yt, yp
        if len(a)>=2 and len(b)>=2:
            corrs.append(np.corrcoef(a,b)[0,1])
        else:
            corrs.append(np.nan)
    k = int(np.nanargmax(corrs))
    return int(lags[k]), float(corrs[k])

for c,name in enumerate(names):
    yt = (y_t[0,c,:,i,j]*y_std[0,c,0,0,0] + y_mean[0,c,0,0,0]).cpu().numpy()
    yp = y_hat_dn[0,c,:,i,j].numpy()
    L, R = best_lag(yt, yp, max_lag=6)
    print(f"{name}: best lag={L} steps, corr={R:.3f}")
    plt.figure(); plt.plot(yt,label="true"); plt.plot(yp,label="pred")
    plt.title(f"{name} @ center pixel (lag {L}, corr {R:.2f})"); plt.legend(); plt.show()


In [4]:
import xarray as xr
ds = xr.open_dataset(r"C:\Users\jr80fd\swin_test\w3ra\Sg_EU.nc")  # works once netCDF4 is present


In [3]:
import xarray as xr
from pathlib import Path

nc_paths = [
    Path(r"C:\Users\jr80fd\swin_test\w3ra\W3RA_Sg_EU.nc"),
    Path(r"C:\Users\jr80fd\swin_test\w3ra\W3RA_Sd_EU.nc"),
    Path(r"C:\Users\jr80fd\swin_test\w3ra\S0_EU.nc"),
]

for p in nc_paths:
    ds = xr.open_dataset(p)
    print("\n", p.name)
    print(ds)            # prints dims/coords/vars
    print("vars:", list(ds.data_vars))
    for v in ds.data_vars:
        print(v, ds[v].dims, ds[v].shape, ds[v].attrs)
    ds.close()


ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html